In [1]:
import torch
import numpy as np
from matplotlib import pyplot as plt
import torch.nn.functional as F

In [2]:
names = open("names.txt", "r").read().splitlines()
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(stoi)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [3]:
def build_dataset(names):
    block_size = 3 #context length of how many chars we take to predict the next char
    X, Y = [],[]
    
    for w in names:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # rolling window, makes sense when you run it
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X,Y

In [47]:
import random
random.seed(42)
random.shuffle(names)
X_train, Y_train = build_dataset(names[:int(0.8*len(names))]) #get first 80% of the dataset
X_valid, Y_valid = build_dataset(names[int(0.8*len(names)):int(0.9*len(names))]) # get between 80%-90%
X_test, Y_test = build_dataset(names[int(0.9*len(names)):]) # get last 10% of dataset

block_size = 3
vocab_size = len(itos)
#[itos[x[2].item()] for x in X_train[:5]]

In [16]:
n_emb = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_emb), generator=g)

#Layer 1
W1 = torch.randn((n_emb * block_size, n_hidden) , generator = g) * (5/3)/(n_emb * block_size)**0.5
b1 = torch.randn(n_hidden, generator = g) * 0.1

#Layer2
W2 = torch.randn((n_hidden, vocab_size) , generator = g) * 0.1
b2 = torch.randn(vocab_size, generator = g) * 0.1

#BatchNorm params
gamma = torch.randn((1, n_hidden)) * 0.1 + 1.0
beta = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b1, gamma, beta]
print(f"number of all params: {sum(p.numel() for p in parameters)}")

number of all params: 4174


In [17]:
batch_size = 32
n = batch_size
# get a mini-batch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator = g)
Xbatch, Ybatch = X_train[ix], Y_train[ix]

In [ ]:
# forward pass, but in smaller steps because i'll need to take gradients of each step

emb = C[ix]
embcat = C[ix]

#Linear layer 1
hpre